Lecture 9 will cover iterators and operator overloading.

Reference
 * [2] Section 9.8-9.10

# Operator overloading special methods

You have seen a few overloaded operators, but there are several important methods.

They are all named according to `__name__` convention (two underscores around the name).

Some (possibly useful) unary operators:
* repr: `repr(x)` (data **repr**esentation)
* len: `len(x)`
* abs: `abs(x)`
* neg: `-a`

and many other things


Common conversions:
* str
* int
* float
* complex
* bool

Lets test out a bunch of these:

In [13]:
class MyClass:
    def __init__(self, x):
        self.x = x

    def __bool__(self):
        return self.x > 0

    def __len__(self):
        return self.x

    def __neg__(self):
        return MyClass(-self.x)

    def __str__(self):
        return "x = " + str(self.x)
    
    def __repr__(self):
        return "MyClass(" + repr(self.x) + ")"

In [17]:
x = MyClass(3)

if x:
    print(x, "has length", len(x))
print([x, -x])
-x

x = 3 has length 3
[MyClass(3), MyClass(-3)]


MyClass(-3)

In [16]:
','.join([str(y) for y in [x, -x]])

'x = 3,x = -3'

### Binary operators

There are a massive amount of binary operators one can overload: `+ * - / // | & << **`.

If is often in the case that both sides of the infix operator has the same type, and the output from the computation is also of the same type, but this is not strictly necessary.

Python will call the suitable special method by doing `a + b === a.__add__(b)`, with similar names for the multitude of other operators (`mul`, `div`, etc.), If `a.__add__` isn't found, then the corresponding "reverse" operation is used: `b.__radd__(a)` (if available).

Many operators have extended assignments as well, and these can be accessed through special methods as well, e.g. `a += b === a.__iadd__(b)`.

In [21]:
import math

class Fraction:
    def __init__(self, num, denom):
        d = math.gcd(num, denom)
        self.num = num // d
        self.denom = denom // d

    def __str__(self):
        return str(self.num) + "/" + str(self.denom)
    
    def __float__(self):
        return self.num / self.denom

    def __add__(self, other):
        # Expecting "other" to be a Fraction.
        # The result is a Fraction as well.
        return Fraction(self.num * other.denom + other.num * self.denom,
                        self.denom * other.denom)
    
    def __iadd__(self, other):
        # self += other
        # We want to modify "self"!
        self.num = self.num * other.denom + other.num * self.denom
        self.denom *= other.denom
        return self # (optional behavior)
        
    def __mul__(self, other):
        return Fraction(self.num * other.num, self.denom * other.denom)

    def __pow__(self, exponent: int):
        # This assumes that the exponent is an integer
        return Fraction(self.num ** exponent, self.denom ** exponent)
    

In [24]:
x = Fraction(3, 4)
y = Fraction(2, 3)

q = y ** 2
x += q
z = (x + y) + x
w = x * y

print(x)
print(float(z))
print(q)
print(w)

43/36
3.0555555555555554
4/9
43/54


** Questions **
* In Python, will `a + b` always give the same result as `b + a`? 
* How about `a * b` and `b * a`?
* Will `a += b` always be the same as `a = a + b`?

In [28]:
a = [1,2,3]
c = a
a += [4,5]
print(c, a)
#a = a + [1,2]
a[0] = 9
print(c, a)

[1, 2, 3, 4, 5] [1, 2, 3, 4, 5]
[9, 2, 3, 4, 5] [9, 2, 3, 4, 5]


### Comparison operators

Comparison operators have a few special rules, because they have no "reversed" operations.
Instead, `a < b` is used like `b > a` when `>`-operation is missing.

Implementing at least one comparison operator from `__lt__`, `__gt__` or `__cmp__` is necessary for comparisons (sort will try them in that order). Typically, `__lt__` is implemented.

In [43]:
class MyTestClass:
    """ A test class where only one instance 
    variable determines the comparison operators
    """
    
    def __init__(self, val, val2):
        self.val = val
        self.val2 = val2

    def __eq__(self, other):
        return abs(self.val) == abs(other.val)

    def __lt__(self, other): # We only check the magnitude:
        return abs(self.val) < abs(other.val)

In [45]:
x = MyTestClass(-34, "Hello")
y = MyTestClass(-34, "Hello")
print(x > y) # Python will look for a match, and will call y < x (which should be mathematically equivalent)
print(x == y)

False
True


If `__eq__` is not implemented, the `is` operator is used instead.

In [7]:
class MyTestClass:
    def __init__(self, val):
        self.val = val

    def __lt__(self, other): # We only check the magnitude:
        return abs(self.val) < abs(other.val)

In [8]:
x = MyTestClass(23)
y = MyTestClass(23)
x == y

False

This strange default behavior is rarely what we want! Therefore, `__eq__` is recommended to overload if `__lt__` has been overloaded.

## Indexing overloading

In [56]:
class LoggingList(list):
    def __setitem__(self, pos, val):
        print("Setting items at:", pos)
        super().__setitem__(pos, val)
        
    def __getitem__(self, pos):
        print("Getting items at:", pos)
        return super().__getitem__(pos)

    def __delitem__(self, pos):
        print("Deleting items at:", pos)
        return super().__delitem__(pos)

In [58]:
x = LoggingList()
x += [1,2,3,4]

x[0:2] = [123, 456]
y = x[2:]
del x[0]

Setting items at: slice(0, 2, None)
Getting items at: slice(2, None, None)
Deleting items at: 0


Note that `a:b:c` is just shorthand for creating a `slice` object.
While this is the most common use for the  `__***item__` methods, this is not at all required, e.g:

In [59]:
class IdentityMatrix:
    def __getitem__(self, row_and_col):
        row, col = row_and_col
        return 1 if row == col else 0

eye = IdentityMatrix()
print(eye[4,5], eye[7,7])

0 1


In [31]:
class CaseInsensitiveDataBase:
    def __init__(self):
        # Some made up data for this example:
        self.dataset = {'A': 7, 'B': 4, 'C': 7, 'D': 0}

    def __getitem__(self, entries):
        return [self.dataset[entry.upper()] for entry in entries]

db = CaseInsensitiveDataBase()
print(db[['b', 'd']])

[4, 0]


## Comment on methods vs operators

This kind of compact notation is often sought by programmers, why have
```python
x.add(y)
```
when you could have
```python
x + y
```

But it's often the case that we are not working with mathematics, but strings, lists, graphical objects, and other types of custom objects.
Therefore, it's often not clear what the operator will actually do (languages like Java disallow this operator overloading for this reason).

When comparing two Car-objects, do we just check if they are of the same model? Same color? Model and color? Same registration number? Top speed?

Operators should not be overused, since it is often more clear to write a descriptive method name. Use it only when the purpose is clear.

# Iterators and generators

In the course so far, we've used multiple examples of the for-loop:

In [1]:
for i in range(5):
    print(i, end=",")

0,1,2,3,4,

In [60]:
for x in "Hello World":
    print(x, end=", ")

H, e, l, l, o,  , W, o, r, l, d, 

In [61]:
for x in zip(range(3), "ABC"):
    print(x, end=", ")

(0, 'A'), (1, 'B'), (2, 'C'), 

All these examples are very different. The `range` keeps generating new numbers until the max value is reached. In the string, we go letter by letter until the end of the string.
The `for`-loop is compact, convenient, and easy to read.

In [62]:
x = range(100000000000000000000000)

### The `for`-loop

Lets assume we have a loop like:
```python
for a in b:
    ...
```
When the `for`-loop starts, it asks for
```python
it = iter(b)    # Looks for b.__iter__()
```
and then it asks for
```python
a = next(it)    # Looks for it.__next__()
```
until a `StopIteration`-exception is raised.

Lets try it out, step-by-step

In [63]:
b = "Hello"
it = iter(b)
print(it)

In [64]:
print( next(it) )
print( next(it) )
print( next(it) )
print( next(it) )
print( next(it) )

H
e
l
l
o


So far so good, what happens when we ask one more time?

In [65]:
next(it)

StopIteration: 

**To sum it up:** 
1. The object needs support the function `iter`
2. Whatever it returned from `iter` must support the function `next`

### Lets create a custom iterator

In [66]:
class CollatzGenerator:
    def __init__(self, n):
        self.n = n

    def __iter__(self):
        # We'll reuse the class generator itself 
        # as the iterator for this simple case
        return self
    
    def __next__(self):
        if self.n == 1:
            raise StopIteration
        if self.n % 2 == 0:
            self.n //= 2
        else:
            self.n = self.n*3 + 1
        
        return self.n

In [69]:
# This should produce 3, 10, 5, 16, 8, 4, 2, 1.
x = CollatzGenerator(6)
for n in x:
    print(n, end=", ")

3, 10, 5, 16, 8, 4, 2, 1, 

Note! In this `CollatzGenerator`, the iterator is the same as the generator itself.
If we try to use `x` again, it will already have reached 1, as the (modified) iterator *is* the generator.

In [70]:
print( x.n )

1


### Another example, the infinite loop!

Lets create a loop over all natural numbers. We would like to have
```python
nat = NaturalNumbers():
for x in nat:
   ...
```
Of course, it's not very good that we modified the starting point of the `nat` (in case we want to reuse it).

We should therefor avoid `def __iter__(self): return self`, and instead return a different iterator.

In [78]:
class zippy:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def __iter__(self):
        class zippy_iter:
            def __init__(self, a_it, b_it):
                self.a_it = a_it
                self.b_it = b_it
            
            def __next__(self):
                return next(self.a_it), next(self.b_it)
            
        return zippy_iter(iter(self.a), iter(self.b))

In [71]:
class NaturalNumbers:
    def __iter__(self):
        # Lets declare an iterator internally:
        class NaturalNumbers_It:
            def __init__(self):
                self.n = 0
       
            def __next__(self):
                self.n += 1
                return self.n
        # Each time we start a for-loop, 
        # we'll return a brand new iterator:
        return NaturalNumbers_It()

In [72]:
nat = NaturalNumbers()

In [77]:
tot = 0
for x in nat:
    tot += x
    # We might have some condition to stop that 
    # doesn't depend on only one the value of x!
    if tot > 1000:
        break
print(tot, x)

1035 45


## Generators

The interested student can also look up the `yield` statement. For example here:
http://www.python-course.eu/python3_generators.php

The for-loop will obtain each `yield`'ed value, and the function will be frozen until the next yielded value is requested:

In [451]:
def countfromto(a, b):
    while a < b:
        print("Yielding a new value!")
        yield a
        a += 1

for i in countfromto(10, 15):
    print(i)

Yielding a new value!
10
Yielding a new value!
11
Yielding a new value!
12
Yielding a new value!
13
Yielding a new value!
14
